In [28]:
#!pip install langchain langchain_community langchain_openai llama_index llama-index-llms-langchain duckduckgo-search

#list package versions for reference:
!pip show langchain langchain_community langchain_openai llama_index llama-index-llms-langchain duckduckgo-search | grep -E "Name:|Version:"


Name: langchain
Version: 0.1.17
Name: langchain-community
Version: 0.0.36
Name: langchain-openai
Version: 0.1.5
Name: llama-index
Version: 0.10.33
Name: llama-index-llms-langchain
Version: 0.1.3
Name: duckduckgo_search
Version: 5.3.0


In [13]:
# Set OpenAI API Key

import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass("Enter OpenAI API Key:")

In [18]:
# Required Imports

from langchain_openai import ChatOpenAI

from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import ChatPromptTemplate

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import ( SimpleDirectoryReader,VectorStoreIndex,StorageContext, load_index_from_storage, Settings)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [19]:
# LLM 
llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0, streaming=True)

# Embedding Model
embed_model = OpenAIEmbedding(
    model="text-embedding-3-small", embed_batch_size=100
)

# Set Llamaindex Configs
Settings.llm = llm 
Settings.embed_model = embed_model

In [21]:
#Download Sample PDFs

!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-05-02 00:14:59--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.1s    

2024-05-02 00:15:00 (16.2 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-05-02 00:15:00--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connec

#### Use LlamaIndex to create Query Engines for individual docs


In [22]:
#Building Indexes for each of the Documents
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
    print("Index was already created. We just loaded it from the local storage.")

except:

    index_loaded = False
    print("Index is not present. We need it to create it again.")

if not index_loaded:

    print("Creating Index..")

    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")
    
    index_loaded = True



#Creating Query engines on top of the indexes

lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

print("LlamaIndex Query Engines created successfully.")

Index is not present. We need it to create it again.
Creating Index..
LlamaIndex Query Engines created successfully.


#### We will now use the LlamaIndex QueryEngineTool abstraction to convert our query engines into Tools which can be used with Agents.

In [23]:
#creating tools for each of our query engines

query_engine_tools = [

    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

### Converting LlamaIndex Tools to Langchain Compatible Format Tools


In [24]:
#before conversion
query_engine_tools

In [25]:
llamaindex_to_langchain_converted_tools = [t.to_langchain_tool() for t in query_engine_tools]

print("No of LlamaIndex Tools:", len(llamaindex_to_langchain_converted_tools))

No of LlamaIndex Tools: 2


In [26]:
#after conversion
llamaindex_to_langchain_converted_tools

[Tool(name='lyft_10k', description='Provides information about Lyft financials for year 2021. Use a detailed plain text question as input to the tool.', args_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, func=<bound method AsyncBaseTool.__call__ of <llama_index.core.tools.query_engine.QueryEngineTool object at 0x1408c6060>>),
 Tool(name='uber_10k', description='Provides information about Uber financials for year 2021. Use a detailed plain text question as input to the tool.', args_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, func=<bound method AsyncBaseTool.__call__ of <llama_index.core.tools.query_engine.QueryEngineTool object at 0x1408c4d10>>)]

#### Define an Additional Langchain Tool for External Web Search Functionality

In [29]:
#Another Langchain Tool

search = DuckDuckGoSearchRun()

duckduckgo_tool = Tool(
        name='DuckDuckGoSearch',
        func= search.run,
        description='Use for when you need to perform an internet search to find information that another tool can not provide.'
)   

langchain_tools = [duckduckgo_tool]

In [30]:
langchain_tools

[Tool(name='DuckDuckGoSearch', description='Use for when you need to perform an internet search to find information that another tool can not provide.', func=<bound method BaseTool.run of DuckDuckGoSearchRun()>)]

#### Combine Tools

In [31]:
#Combine to create final list of tools
tools = llamaindex_to_langchain_converted_tools + langchain_tools

print("No of Final tools:", len(tools))

No of Final tools: 3


In [32]:
tools

[Tool(name='lyft_10k', description='Provides information about Lyft financials for year 2021. Use a detailed plain text question as input to the tool.', args_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, func=<bound method AsyncBaseTool.__call__ of <llama_index.core.tools.query_engine.QueryEngineTool object at 0x1408c6060>>),
 Tool(name='uber_10k', description='Provides information about Uber financials for year 2021. Use a detailed plain text question as input to the tool.', args_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, func=<bound method AsyncBaseTool.__call__ of <llama_index.core.tools.query_engine.QueryEngineTool object at 0x1408c4d10>>),
 Tool(name='DuckDuckGoSearch', description='Use for when you need to perform an internet search to find information that another tool can not provide.', func=<bound method BaseTool.run of DuckDuckGoSearchRun()>)]

#### Inititalize Langchain's Tool Calling Agent with an Agent Executor


By creating the 'tool calling agent', we provide access to multiple tools for the LLM. The agent intelligently chooses to repeatedly call the appropriate tools and receives results until the user's query is resolved. 

The tool-calling agent is the newest, and more general agent implementation that supports a wide variety of LLM providers such as OpenAI, Anthropic, Google Gemini, Mistral, and others.



In [33]:
system_context = "You are a stock market expert.\
You will answer questions about Uber and Lyft companies as in the persona of a veteran stock market investor."

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_context,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt,)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'agent_scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a stock market expert.You will answer qu

In [34]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True, handle_parsing_errors=True, max_iterations=10)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'agent_scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_vari

### Testing:

In [35]:
question =  "What was Lyft's revenue growth in 2021?"

response = agent_executor.invoke({"input": question})

print("\nFinal Response:", response['output'])



> Entering new AgentExecutor chain...

Invoking: `lyft_10k` with `{'input': "What was Lyft's revenue growth in 2021?"}`




/Users/chirag/anaconda3/envs/GenAiMay24/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Lyft's revenue increased 36% in 2021 compared to the prior year.Lyft's revenue grew by 36% in 2021 compared to the previous year.

> Finished chain.

Final Response: Lyft's revenue grew by 36% in 2021 compared to the previous year.


In [36]:
#It invoked the lyft query engine tool as desired.

In [37]:
question =  "Is Uber profitable?"

response = agent_executor.invoke({"input": question})

print("\nFinal Response:", response['output'])



> Entering new AgentExecutor chain...

Invoking: `uber_10k` with `{'input': "What was Uber's profitability in 2021?"}`


In 2021, Uber Technologies, Inc. reported a net loss attributable to the company of $496 million. Despite this net loss, the company saw an improvement in its profitability compared to the previous year, with the net loss decreasing by 93% from $6,768 million in 2020. Additionally, Uber's Adjusted EBITDA loss was $774 million in 2021, which was an improvement of $1.8 billion from the previous year.Uber was not profitable in 2021, as the company reported a net loss attributable to Uber of $496 million. However, it's important to note that this was a significant improvement compared to the previous year, with the net loss decreasing by 93% from $6,768 million in 2020. Additionally, Uber's Adjusted EBITDA loss was $774 million in 2021, which also showed an improvement of $1.8 billion from the previous year. This indicates that while Uber was not profitable in 2021, th

In [38]:
#It invoked the uber query engine tool as desired.

In [39]:
question =  "List me the names of Uber's board of directors."

response = agent_executor.invoke({"input": question})

print("\nFinal Response:", response['output'])





> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearch` with `Uber board of directors 2023`


Fourth quarter trips and monthly active platform consumers grew 24% and 15% year-over-year, respectively Fourth quarter Gross Bookings grew 22% year-over-year and 21% year-over-year on a constant currency basis Fourth quarter net income of $1.4 billion; Income from operations of $652 million; Record Adjusted EBITDA margin Fourth quarter operating cash flow of $823 million; Free cash flow of ... On November 7, 2023, Uber Technologies, Inc. (the "Company") issued a press release announcing its financial results for the third quarter ended September 30, 2023. A copy of the press release is furnished herewith as Exhibit 99.1 and is incorporated herein by reference. ... Election of Directors; Appointment of Certain Officers ... August 15, 2023 05:01 AM Eastern Daylight Time. OAKLAND, Calif.-- ( BUSINESS WIRE )--Today, social impact venture Him For Her announced Dara Khosrowshahi, CEO 

In [40]:
#Since this information is absent in any of the documents, it invoked the external search tool.